In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask ,redirect ,url_for ,request , render_template
import librosa
import tensorflow as tf
import numpy as np
import sqlite3 

SAVED_MODEL_PATH = "speaker/model.h5"
SAMPLES_TO_CONSIDER = 22050*3
model = tf.keras.models.load_model(SAVED_MODEL_PATH)
#path="speaker/speaker_1_4_2.wav"



def preprocess( file_path, num_mfcc=13, n_fft=2048, hop_length=512):
  """Extract MFCCs from audio file.
  :param file_path (str): Path of audio file
  :param num_mfcc (int): # of coefficients to extract
  :param n_fft (int): Interval we consider to apply STFT. Measured in # of samples
  :param hop_length (int): Sliding window for STFT. Measured in # of samples
  :return MFCCs (ndarray): 2-dim array with MFCC data of shape (# time steps, # coefficients)
        """

  # load audio file
  signal, sample_rate = librosa.load(file_path)

  if len(signal) >= SAMPLES_TO_CONSIDER:
  # ensure consistency of the length of the signal
    signal = signal[:SAMPLES_TO_CONSIDER]

            # extract MFCCs
    MFCCs = librosa.feature.mfcc(signal, sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                         hop_length=hop_length)
    return MFCCs.T

def predict( file_path):
  """
  :param file_path (str): Path to audio file to predict
  :return predicted_keyword (str): Keyword predicted by the model
  """

  # extract MFCC
  MFCCs = preprocess(file_path) 

  # we need a 4-dim array to feed to the model for prediction: (# samples, # time steps, # coefficients, 1)
  MFCCs = MFCCs[np.newaxis, ..., np.newaxis]

  # get the predicted label
  predictions = model.predict(MFCCs)
  predicted_index = np.argmax(predictions)
  #print(predicted_index)
  return predicted_index

app = Flask(__name__)
@app.route('/')  
def upload():  
    return render_template("file_upload_form.html")  
 
@app.route('/success', methods = ['POST'])  
def success():  
    if request.method == 'POST':  
        f = request.files['file']  
        f.save(f.filename)  
        t=predict(f.filename)
        return redirect(url_for("view",usr=t+1))
        #return render_template("success.html", name = t) 

@app.route("/view/<usr>")  
def view(usr):  
    #t=1
    con = sqlite3.connect("employee.db")  
    con.row_factory = sqlite3.Row  
    cur = con.cursor()  
    #cur.execute("select * from Employees")
    cur.execute("select * from Employees where id = {}".format(usr))  
    rows = cur.fetchall()  
    return render_template("A_view.html",rows = rows)
    

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

C:\Users\Rajansahu\Anaconda3\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\Rajansahu\Anaconda3\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/May/2020 08:38:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2020 08:38:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [1